<a href="https://colab.research.google.com/github/Navarra2/ML/blob/master/CLAProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random, numpy as np

In [292]:
def alg1(A,u,f,eps,maxit):
  # i didn't test anything here
  "Algorithm 1 with Gauss-Radau quadrature"

  #find the values a and b bounds of the spectrum using the Gerschgorin circles
  a=np.inf
  b =-np.inf
  for i in range(np.shape(A)[0]):
    radius = 0
    for j in range(np.shape(A)[1]):
      if j==i:
        radius+=0
      else:
        radius += np.abs(A[i][j])
    a_candidate = np.abs(A[i][i]) - radius
    b_candidate = np.abs(A[i][i]) + radius

    if a_candidate<a:
      a = a_candidate
    if b_candidate > b:
      b = b_candidate
  #as said in the paper, if the value of a is less or equal to zero, set it to ~10^-4    
  if a<=0:
    a = 10**(-4)
  #beginning of the Algorithm
  x_old = np.zeros_like(u)
  x_new = u/np.linalg.norm(u)
  gamma = np.zeros(maxit)
  alpha = np.zeros(maxit)
  Ia_old = -np.inf
  Ia_new = np.inf
  Ib_old = -np.inf
  Ib_new = np.inf
  j = 1
  while (np.abs(Ia_old - Ia_new)>eps or np.abs(Ib_old - Ib_new)>eps) and j< maxit:
    #getting the values of alpha, gamma, r
    Ax = np.matmul(A,x_new)
    alpha[j] = np.matmul(x_new.transpose(),Ax)
    r_new = Ax - alpha[j]*x_new - gamma[j-1]*x_old
    gamma[j] = np.linalg.norm(r_new)

    #construction of T^tilde
    Tj = np.diag(gamma[1:j],-1) + np.diag(alpha[1:j+1],0) + np.diag(gamma[1:j],1)
    ej = np.identity(j)[:,-1]

    #check if the matrices are non singular (this will not happen in general due to the interlacing property of the Tj's in A)
    Ta =Tj - a*np.identity(j)
    Tb = Tj - b*np.identity(j)

    if np.linalg.det(Ta)==0 or np.linalg.det(Tb)==0:
      break

    deltaa = np.linalg.solve(Ta,ej)
    deltab = np.linalg.solve(Tb,(gamma[j]**2)*ej)

    phia = deltaa[-1] + a
    phib = deltab[-1] + b

    ej = np.expand_dims(ej, axis=0)    
    Tj_tildea = np.block([
    [Tj,               ej.transpose()],
    [ej, phia               ]
])  
    Tj_tildeb = np.block([
    [Tj,               ej.transpose()],
    [ej, phib               ]
])  

    Ia_old = Ia_new
    Ib_old = Ib_new
    #compute I_j
    Ia_new = gauss_radau(Tj_tildea,f,a,b)
    Ib_new = gauss_radau(Tj_tildeb,f,a,b)
    j+=1

  #Compute the estimates for lower and upper bounds
  U = Ia_new*np.linalg.norm(u)**2
  L = Ib_new*np.linalg.norm(u)**2
  if j>= maxit:
    print('exceeded the number of iterations')
  if L<U:
    return [L,U]
  else:
    return [U,L]


In [31]:
def gauss_radau(T,f,a,b): #works
  teta, eigvec= np.linalg.eig(T)
  w = np.zeros_like(teta)
  I=0
  #normalize them to get w
  for i in range(np.shape(eigvec)[1]):
    vec = eigvec[:,i] 
    wi = (vec/np.linalg.norm(vec))[0]**2
    I += wi*f(teta[i])

  return I


In [308]:
def alg2(A,m,p,eps,maxit):
#computes an unbiased estimator I_p of tr(A) and a confidence interval (L_p, U_p) with a user probability
  n = len(A)
  Lmin = np.inf
  Umax = -np.inf
  sumLj = 0
  sumUj = 0

  for j in range(m):
    zj = np.random.random_sample(n)
    for i in range(n):
      if zj[i]<0.5:
        zj[i] = 0
      else:
        zj[i] = 1
    #to avoid division by 0 we treat the case zj=[0,...,0] separately    
    if zj.all()==0:
      S=[0,0]
    else:
      S = alg1(A,zj,f,eps,50)
    sumLj =sumLj + S[0]
    sumUj =sumUj + S[1]
    Ij = (sumUj + sumLj)/2/(j+1)

    Lmin = min(Lmin,S[0])
    Umax = max(Umax,S[1])

    eta2 = -0.5*j*(Umax-Lmin)**2*np.log(1-p)/2
    Lpj = (sumLj - np.sqrt(eta2))/(j+1)
    Upj = (sumUj +np.sqrt(eta2))/(j+1)

  Ip = Ij
  Lp = Lpj
  Up = Upj

  return [Ip,Lp,Up]

In [307]:
# small experiments
def f(x):
  return 1/x
f(2)
A = [[1,0,0],[0,2,1],[0,1,1]]
B = [[1,0,0],[0,2,1],[0,1,1]]
eps = 0.01
maxit = 50
u=[0,1,1]
alg1(B,u,f,eps,maxit)


[0.8348486239936346, 0.8352715568918101]

In [277]:
#small experiments
A = [[1,1,1],[0,2,1],[0,1,1]]
m=10
alg2(A,m,0.5,eps,maxit)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


[0.1162386925868641, -0.032214918106395735, 0.2905231238549827]

Questions to argue with Leelo:
-Apply twice so we can have a lower bound and an upper bound.
- In algorithm 2 we need to take out the cases when there are inf's or Nan's, or if the matrix is singular
- What to do to the cases that exceed the number of iterations?

In [349]:
#Matrices needed for the Results of Paper 1
#Linear Heat Flow
k = 3
v = 1
D = np.diag((1+4*v)*np.ones(k))+np.diag(-v*np.ones(k-1),1)+np.diag(-v*np.ones(k-1),-1)
C = np.diag(-v*np.ones(k-1),1)+np.diag(-v*np.ones(k-1),-1)

LHF = np.kron(D,np.eye(C.shape[0],C.shape[1])) + np.kron(np.eye(D.shape[0],D.shape[1]),C)

#Vicsek fractal Hamiltonian
#Poisson Matrix
#Wathen Matrix
#Lehmer Matrix
#Pei Matrix